In [1]:
import pandas as pd
import os
import csv
import uuid
from datetime import datetime

In [2]:
df = pd.read_csv('items.csv')
df.columns = df.columns.str.strip()
df.rename(columns={"Avg. Price (₹)": "Price"}, inplace=True)
df

,Sno,Item,Price
0,1,Rice,70
1,2,Bread,48
2,3,Eggs,8
3,4,BeetRoot,20
4,5,Milk,39
5,6,Apples,45
6,7,Tomatoes,35
7,8,Cheese,40
8,9,Potatoes,37
9,10,Bananas,10


In [3]:
cart = {}
save_for_later = {}
def additem():
    print("\nAvailable Items:")
    print("Item\t\tPrice (₹)")
    print("-" * 20)
    for index, row in df.iterrows():
        print(f"{row['Item']:<15} ₹{row['Price']}")
    print("\nInstructions:")
    print("- Type item name to add to cart")
    print("- Type 'save [item_name] [qty]' to save item for later")
    print("- Type 'done' to finish\n")
    while True:
        entry = input("Add item: ").strip()
        if entry.lower() == 'done':
            break
        elif entry.lower().startswith("save "):
            parts = entry.split()
            if len(parts) == 3 and parts[1] in df['Item'].values and parts[2].isdigit():
                item = parts[1]
                qty = int(parts[2])
                save_for_later[item] = qty
                print(f"💾 Saved for later: {item} - Quantity: {qty}")
            else:
                print("Invalid save format. Use: save <item_name> <quantity>")
        elif entry.lower() in df['Item'].str.lower().values:
            matched_item = df.loc[df['Item'].str.lower() == entry.lower(), 'Item'].values[0]
            while True:
                qty_input = input(f"Enter quantity for '{entry}': ").strip()
                if qty_input.isdigit():
                    quantity = int(qty_input)
                    break
                else:
                    print("Please enter a valid integer quantity.")
            cart[matched_item] = cart.get(matched_item, 0) + quantity
            print(f"✅ Added: {matched_item} - Quantity: {cart[matched_item]}")
        else:
            print(f"{matched_item} not found in item list. Please try again.")

In [4]:
def update_cart():
    print("\n🛒 Current Cart:")
    if not cart:
        print("Cart is empty.")
    else:
        for item, qty in cart.items():
            print(f"{item:<15} {qty}")
    if save_for_later:
        print("\n💾 Saved for Later:")
        for item, qty in save_for_later.items():
            print(f"{item:<15} {qty}")
    print("\nYou can update quantities or move saved items to cart (type 'done' to finish):")
    while True:
        item_name = input("Enter item to update/add (or 'done'): ").strip()
        if item_name.lower() == 'done':
            break
        elif item_name in df['Item'].values:
            qty_input = input(f"Enter quantity for '{item_name}' (0 to remove): ").strip()
            if qty_input.isdigit():
                new_qty = int(qty_input)
                if new_qty == 0:
                    if item_name in cart:
                        del cart[item_name]
                        print(f"❌ Removed {item_name} from cart.")
                    elif item_name in save_for_later:
                        del save_for_later[item_name]
                        print(f"❌ Removed {item_name} from saved items.")
                    else:
                        print(f"{item_name} not found.")
                else:
                    cart[item_name] = new_qty
                    if item_name in save_for_later:
                        del save_for_later[item_name]
                        print(f"✅ Moved {item_name} from saved to cart.")
                    print(f"✅ Updated {item_name} to quantity {new_qty}")
            else:
                print("Invalid quantity.")
        else:
            print(f"{item_name} not found in item list.")

In [5]:
def calculate_total():
    global transaction_id, invoice_data, final_total
    if not cart:
        print("Cart is empty.")
        return 0
    subtotal = 0
    for item, qty in cart.items():
        price = int(df.loc[df['Item'] == item, 'Price'].values[0])
        subtotal += price * qty
    tax_10_percent = round(subtotal * 0.1, 2)
    gov_tax = 3
    final_total = subtotal + tax_10_percent + gov_tax
    transaction_id = str(uuid.uuid4())[:8]
    current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    invoice_data = {
        'Transaction ID': transaction_id,
        'Customer Name': customer_name,
        'Date Time': current_time,
        'Subtotal': subtotal,
        'Tax 10%': tax_10_percent,
        'Gov Tax': gov_tax,
        'Total Amount': final_total,
        'Items': '; '.join([f"{item} x{qty}" for item, qty in cart.items()])
    }
    # Save invoice
    file_exists = os.path.isfile('invoice_records.csv')
    with open('invoice_records.csv', 'a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=invoice_data.keys())
        if not file_exists:
            writer.writeheader()
        writer.writerow(invoice_data)
    return final_total

In [6]:
def display_invoice():
    print("\n🧾 INVOICE RECEIPT")
    print("=" * 60)
    print("FINAL INVOICE")
    print("=" * 60)

    print("Customer Name:", customer_name)
    print("Transaction ID:", invoice_data['Transaction ID'])
    print("Date & Time:", invoice_data['Date Time'])

    print("-" * 60)
    print("Item   Qty   Price (₹)   Total (₹)")
    print("-" * 60)

    for item, qty in cart.items():
        price = int(df.loc[df['Item'] == item, 'Price'].values[0])
        item_total = price * qty
        print(item, qty, price, item_total)

    print("-" * 60)
    print("Subtotal:", "₹" + str(invoice_data['Subtotal']))
    print("Tax (10%):", "₹" + str(invoice_data['Tax 10%']))
    print("Gov. Tax:", "₹" + str(invoice_data['Gov Tax']))

    print("=" * 60)
    print("TOTAL PAID:", "₹" + str(invoice_data['Total Amount']))
    print("=" * 60)

    print("\n✅ Thank you for shopping with us! Have a nice day! ✅\n")

In [9]:
customer_name = input("\nEnter customer name: ").strip()
print(f"Welcome, {customer_name}!")
additem()
update_cart()
total_amount = calculate_total()
if total_amount == 0:
    print("\n❌ No items in cart. Cannot proceed with payment or invoice.")
else:
    pay = input("\nWould you like to proceed with the payment? (yes/no): ").strip().lower()
    if pay == 'yes':
        print(f"\nProcessing your payment of ₹{total_amount}...")
        print("💰 Payment successful! ✅")
        display_invoice()
        feedback_prompt = input("\nWould you like to provide feedback? (yes/no): ").strip().lower()
        if feedback_prompt == 'yes':
            feedback = input("Please enter your feedback: ").strip()
        else:
            feedback = "Not given any feedback"
        feedback_data = {
            'Transaction ID': transaction_id,
            'Customer Name': customer_name,
            'Feedback': feedback
        }
        file_exists = os.path.isfile('feedback_records.csv')
        with open('feedback_records.csv', 'a', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=feedback_data.keys())
            if not file_exists:
                writer.writeheader()
            writer.writerow(feedback_data)

        print("\n✅ Feedback recorded successfully!")
        print(f"\nThank you for visiting, {customer_name}! Have a great day! 😊")
    else:
        print("\n❌ Payment was not completed. Invoice and feedback will not be recorded.")

Welcome, ram!

Available Items:
Item		Price (₹)
--------------------
Rice            ₹70
Bread           ₹48
Eggs            ₹8
BeetRoot        ₹20
Milk            ₹39
Apples          ₹45
Tomatoes        ₹35
Cheese          ₹40
Potatoes        ₹37
Bananas         ₹10

Instructions:
- Type item name to add to cart
- Type 'save [item_name] [qty]' to save item for later
- Type 'done' to finish

✅ Added: Apples - Quantity: 4

🛒 Current Cart:
Apples          4

You can update quantities or move saved items to cart (type 'done' to finish):

Processing your payment of ₹201.0...
💰 Payment successful! ✅

🧾 INVOICE RECEIPT
FINAL INVOICE
Customer Name: ram
Transaction ID: 99f3e2ad
Date & Time: 2025-07-07 10:13:57
------------------------------------------------------------
Item   Qty   Price (₹)   Total (₹)
------------------------------------------------------------
Apples 4 45 180
------------------------------------------------------------
Subtotal: ₹180
Tax (10%): ₹18.0
Gov. Tax: ₹3
TOTAL PAI